In [1]:
source("Classes.R")

DEFAULT_CR <- 0.5
DEFAULT_F <- 0.8

is_empty <- function(x) return(length(x) ==0 )

DifferentialEvolution <- setRefClass(
    "DifferentialEvolution",
    contains = "Algorithm",
    fields = list(
        cr = "numeric",
        f = "numeric",
        H = "data.frame" 
    ),
    methods = list(
        init_default_parameters = function(){
            if(is_empty(f)){
                f <<- DEFAULT_F
            }
            if(is_empty(cr)){
                cr <<- DEFAULT_CR
            }
        },
        crossover = function(x,y){
            z <- c()
            for (i in 1:length(x)){
                a <- runif(1, 0, 1)
                if (a < cr) 
                z <- append(z, y[i])
                else 
                z <- append(z, x[i])
            }
            return(z)
        },

        tournanemt = function(x,y){
#             print(x)
#             print(y)
            cond <- x['fitness'] > y['fitness']
            if(cond) 
                return(y)
            else
                return(x)
        },

        run = function() {
            init_default_parameters()
            gen_random_population()
            H <<- population
            mi <- dim(population)[1]
            iterations <<- 0
            while(iterations < 200){
                next_P <- population
                for (i in 1:mi){
                    indexes <- sample(1:mi, 3, replace=F)
                    r <- as.matrix(population[indexes[1],-length(population)])
                    d_e <- as.matrix(population[indexes[2:3],-length(population)])
                    
                    M <- r + f*(d_e[2,] - d_e[1,])
                    cross <- crossover(r, M)
#                     print(sprintf("d_e: %s", d_e))
#                     print(sprintf("M: %s", M))
#                     print(sprintf("r: %s", r))
#                     print(sprintf("cross: %s", cross))
#                     print('------------')
#                     print(objective_fun$evaluate(cross))
                    O <- data.frame(t(cross), fitness=objective_fun$evaluate(cross))
                    append(H,O)
                
                    next_P[i,] <- tournanemt(population[i,], O)
                }
                population <<- next_P
                print(sel_best())
                print(objective_fun$evaluate(sel_best()))
                iterations <<- iterations+1
            }
        }
    )
)

In [2]:
sum_of_squares<-function(point){
  sum <- 0.0
  for(p in point)
  {
    sum <- sum + p^2
  }
  return(sum)
}

of <- ObjectiveFunction$new(fun=sum_of_squares, dim=3)
alg <- DifferentialEvolution$new(objective_fun=of, point_dim=of$dim)

In [3]:
alg$run()

[1] -0.0408437  0.3176092  0.2693130
[1] 0.1750733
[1] -0.0408437  0.3176092  0.2693130
[1] 0.1750733
[1]  0.06029634  0.01080525 -0.18242361
[1] 0.03703078
[1] -0.13438161  0.05667083 -0.11394296
[1] 0.034253
[1] -0.13438161  0.05667083 -0.11394296
[1] 0.034253
[1] -0.13438161  0.05667083 -0.11394296
[1] 0.034253
[1] -0.06827577 -0.03656574  0.05971713
[1] 0.00956477
[1] -0.06827577 -0.03656574 -0.03023764
[1] 0.006912949
[1] -0.06827577 -0.03656574 -0.03023764
[1] 0.006912949


ERROR: Error in sel_best(): 'list' object cannot be coerced to type 'double'


In [82]:
class(as.matrix(alg$population[,-length(alg$population)]))

[1] "matrix" "array"

In [25]:
library(pracma)

In [26]:
P <- read.csv(".\\temp.csv", header = FALSE)
population <- as.matrix(P)

In [27]:
population

V1,V2
16.74117,19.28437
17.66120,16.05978
15.70451,15.32761
18.41005,19.12274
18.05333,19.31244
12.40951,13.06823
16.46795,18.14886
16.83206,13.76698
18.88690,12.46731
10.97053,18.65538


In [28]:
sum_of_squares<-function(point){
  sum <- 0.0
  for(p in point)
  {
    sum <- sum + p^2
  }
  return(sum)
}

In [29]:
# static parameters init
counteval <- 200
population_size <- dim(population)[1]
point_dim <- dim(population)[2]
m <- runif(point_dim)
sigma <- 0.3 
lambda <- 4 + floor(3*log(point_dim))
mu <- lambda/2
weights <- log(mu+1/2)-log(1:mu)
mu <- floor(mu)
weights <- weights/sum(weights)
mueff <- sum(weights)^2/sum(weights^2)
cc <-  (4+mueff/point_dim) / (point_dim+4 + 2*mueff/point_dim)
cs <-  (mueff+2) / (point_dim+mueff+5)
c1 <- 2 / ((point_dim+1.3)^2+mueff)
cmu <- min(1-c1, 2 * (mueff-2+1/mueff) / ((point_dim+2)^2+mueff))
damps <- 1 + 2*max(0, sqrt((mueff-1)/(point_dim+1))-1) + cs

In [30]:
# dynamic parameters init
pc <- numeric(point_dim)
ps <- numeric(point_dim)
B <- eye(point_dim)                       
D <- ones(point_dim , 1 ) 
# C <- B %*% diag(D^2) %*% t(B)           
# invsqrtC <- B %*% diag(D^(-1)) %*% t(B)   
C <- eye(point_dim)
invsqrtC <- eye(point_dim)

eigeneval <- 0                    
chiN <- point_dim^0.5*(1-1/(4*point_dim)+1/(21*point_dim^2))

In [31]:
generated_pop = m + sigma * MASS::mvrnorm(population_size, mu = numeric(point_dim), Sigma = C)

In [32]:
fitnesses = c()
for (i in 1:dim(generated_pop)[1]){
    point <- generated_pop[i,]
    fitness <- sum_of_squares(point)
    fitnesses <- append(fitnesses, fitness)
}
fitnesses

[1] 0.08508779 0.63738044 0.10377316 0.04684076 0.89762696 0.16372379
 [7] 0.28775860 0.39230143 0.40849446 0.24459653 0.14247662 0.06335395
[13] 0.02156110 0.40459520 0.02100881 0.53228653 0.50586014 0.43417967
[19] 0.37301064 0.13478060 0.14495620 0.61149392 0.01358507 0.24363702
[25] 0.63714105 0.56867590 0.28140553 0.02212954 0.13726525 0.16170884

In [33]:
generated_pop_sorted <- generated_pop[order(fitnesses, decreasing=FALSE),]

In [34]:
t(generated_pop_sorted)[,1:mu]

-0.11009847,0.06493569,-0.05721378
-0.03825439,0.12958461,0.13523195


In [74]:
new_m <- t(generated_pop_sorted)[,1:mu] %*% weights
new_m

0.02055006
-0.03144238


In [75]:
ps <- (1-cs)*ps + sqrt(cs*(2-cs)*mueff) * invsqrtC %*% (new_m-m) / sigma;

In [76]:
ps

-0.8807763
-2.5411862


In [77]:
hsig = norm(ps)/sqrt(1-(1-cs)^(2*counteval/lambda))/chiN < 1.4 + 2/(point_dim+1)

In [78]:
pc <- (1-cc)*pc + hsig * sqrt(cc*(2-cc)*mueff) * (new_m-m) / sigma

In [79]:
pc

0
0


In [81]:
artmp <- (1/sigma) * (t(generated_pop_sorted)[,1:mu] - matrix(rep(m,mu), point_dim, mu))

In [87]:
t(generated_pop_sorted)[,1:mu]
t(generated_pop_sorted)
matrix(rep(m,mu), point_dim, mu)
m

-0.02215647,0.05552704,0.24064310
-0.05396757,0.02649575,-0.05871676


-0.02215647,0.05552704,0.24064310,0.0919708,0.3117240,0.2708393,0.02293589,0.3645684,0.2898516,0.40635471,...,-0.5323960,0.4207894,0.8108636,0.6642374,0.9201887,0.8425667,0.8358435,0.1280729,0.7284329,0.8645903
-0.05396757,0.02649575,-0.05871676,0.2840969,0.0410628,0.2083261,0.37218851,-0.1532404,-0.2715714,0.07016288,...,0.7526016,0.8414252,0.5055106,0.8087036,0.5249749,0.6567264,0.7903590,1.1511380,0.9761941,0.8596116


0.2433543,0.2433543,0.2433543
0.6113848,0.6113848,0.6113848


[1] 0.2433543 0.6113848

In [82]:
artmp

-0.8850358,-0.6260907,-0.009037199
-2.2178412,-1.9496301,-2.233671864


In [88]:
C <- (1-c1-cmu) * C + c1*(pc%*%t(pc) + (1-hsig)*cc*(2-cc)*C) + cmu * artmp %*% diag(weights) %*% t(artmp)

In [89]:
C

0.95564370,0.09253824
0.09253824,1.18683234


In [28]:
sigma <- sigma * exp((cs/damps)*(norm(ps)/chiN - 1))
sigma

[1] 0.4221412

# ----------------

In [55]:
library(pracma)

P <- read.csv("C:\\Users\\Tata\\Desktop\\OptimizationAlgorithms\\R_shallow\\data.csv", header = FALSE)
population <- as.matrix(P)

sum_of_squares<-function(point){
  sum <- 0.0
  for(p in point)
  {
    sum <- sum + p^2
  }
  return(sum)
}

# static parameters init
counteval <- 200
population_size <- dim(population)[1]
point_dim <- dim(population)[2]
m <- runif(point_dim)
sigma <- 0.3 
lambda <- 4 + floor(3*log(point_dim))
mu <- lambda/2
weights <- log(mu+1/2)-log(1:mu)
mu <- floor(mu)
weights <- weights/sum(weights)
mueff <- sum(weights)^2/sum(weights^2)
cc <-  (4+mueff/point_dim) / (point_dim+4 + 2*mueff/point_dim)
cs <-  (mueff+2) / (point_dim+mueff+5)
c1 <- 2 / ((point_dim+1.3)^2+mueff)
cmu <- min(1-c1, 2 * (mueff-2+1/mueff) / ((point_dim+2)^2+mueff))
damps <- 1 + 2*max(0, sqrt((mueff-1)/(point_dim+1))-1) + cs

# dynamic parameters init
pc <- numeric(point_dim)
ps <- numeric(point_dim)
B <- eye(point_dim)                       
D <- ones(point_dim , 1 ) 
# C <- B %*% diag(D^2) %*% t(B)           
# invsqrtC <- B %*% diag(D^(-1)) %*% t(B)   
C <- eye(point_dim)
invsqrtC <- eye(point_dim)

eigeneval <- 0                    
chiN <- point_dim^0.5*(1-1/(4*point_dim)+1/(21*point_dim^2))

t <- 0
while(t < 200){
  t <- t+1
  generated_pop <- m + sigma * MASS::mvrnorm(population_size, mu = numeric(point_dim), Sigma = C)

  fitnesses <- c()
  for (i in 1:dim(generated_pop)[1]){
      point <- generated_pop[i,]
      fitness <- sum_of_squares(point)
      fitnesses <- append(fitnesses, fitness)
  }

  generated_pop_sorted <- generated_pop[order(fitnesses, decreasing=FALSE),]
  new_m <- t(generated_pop_sorted)[,1:mu] %*% weights
  ps <- (1-cs)*ps + sqrt(cs*(2-cs)*mueff) * invsqrtC %*% (new_m-m) / sigma
  hsig <- norm(ps)/sqrt(1-(1-cs)^(2*counteval/lambda))/chiN < 1.4 + 2/(point_dim+1)
  pc <- (1-cc)*pc + hsig * sqrt(cc*(2-cc)*mueff) * (new_m-m) / sigma
  artmp <- (1/sigma) * (t(generated_pop_sorted)[,1:mu] - matrix(rep(m,mu), point_dim, mu))
  C <- (1-c1-cmu) * C + c1*(pc%*%t(pc) + (1-hsig)*cc*(2-cc)*C) + cmu * artmp %*% diag(weights) * t(artmp)
  sigma <- sigma * exp((cs/damps)*(norm(ps)/chiN - 1))
 
  print(sum_of_squares(generated_pop_sorted[1,]))
    
C_upper <- C
C_upper_diag <- C
u <- lower.tri(C, diag = TRUE)
u_d <-  lower.tri(C, diag = FALSE)
C_upper[u] <- 0
C_upper_diag[u_d] <- 0
C <- C_upper_diag + t(C_upper)
    eig <- eigen(C)
D <- eig$values
B <- eig$vectors
    D <- sqrt(D)
    invsqrtC <- B %*% diag(D^-1) %*% t(B)
     m <- c(new_m)
}

[1] 0.04878
[1] 0.1982859
[1] 0.1690439
[1] 0.2516478
[1] 0.03250462
[1] 0.1913589
[1] 0.07449304
[1] 0.05550102
[1] 0.005311501
[1] 0.0293127
[1] 0.01245349
[1] 0.003542674
[1] 0.001924882
[1] 0.0002697387
[1] 0.001163518
[1] 0.00100622
[1] 0.0003071322
[1] 0.000169734
[1] 0.0001107777
[1] 0.0001345023
[1] 0.0001265208
[1] 1.224117e-05
[1] 1.4234e-05
[1] 8.558894e-06
[1] 8.957593e-06
[1] 1.250779e-06
[1] 5.499186e-06
[1] 1.284582e-06
[1] 3.956841e-07
[1] 3.566075e-07
[1] 2.367838e-07
[1] 2.749106e-07
[1] 9.54554e-08
[1] 1.907025e-07
[1] 5.347541e-08
[1] 1.488871e-08
[1] 2.234067e-09
[1] 2.307653e-08
[1] 4.996297e-09
[1] 6.102071e-09
[1] 2.708391e-09
[1] 1.01069e-08
[1] 4.279934e-09
[1] 4.828399e-10
[1] 1.204365e-10
[1] 6.399081e-10
[1] 2.490797e-10
[1] 2.074458e-10
[1] 1.368509e-10
[1] 1.393294e-10
[1] 9.884728e-11
[1] 1.647412e-11
[1] 6.455718e-12
[1] 1.084101e-12
[1] 1.615331e-11
[1] 2.139649e-12
[1] 6.836964e-13
[1] 6.170649e-13
[1] 1.037128e-12
[1] 1.00576e-12
[1] 6.583939e-13
[1]

In [31]:
C[lower.tri(C)]

[1]  0.049761024 -0.013252822 -0.004654042

In [30]:
C

0.93263193,0.024880512,-0.002750209
0.04976102,1.036226731,-0.001931602
-0.01325282,-0.004654042,0.931625558


In [34]:
C_upper <- C
C_upper_diag <- C
u <- lower.tri(C, diag = TRUE)
u_d <-  lower.tri(C, diag = FALSE)
C_upper[u] <- 0
C_upper_diag[u_d] <- 0
C_upper
C_upper_diag

0,0.02488051,-0.002750209
0,0.00000000,-0.001931602
0,0.00000000,0.000000000


0.9326319,0.02488051,-0.002750209
0.0000000,1.03622673,-0.001931602
0.0000000,0.00000000,0.931625558


In [37]:
new_C <- C_upper_diag + t(C_upper)

In [38]:
eigen(new_C)

eigen() decomposition
$values
[1] 1.0419489 0.9324878 0.9260476

$vectors
            [,1]       [,2]       [,3]
[1,] -0.22240839 -0.3635553  0.9046337
[2,] -0.97469139  0.1044306 -0.1976637
[3,]  0.02260978  0.9257007  0.3775804


In [44]:
eig <- eigen(new_C)
D <- eig$values
B <- eig$vectors

In [46]:
D <- sqrt(D)

[1] 1.0207590 0.9656541 0.9623137

In [51]:
invsqrtC <- B %*% diag(D^-1) %*% t(B)

In [64]:
t(generated_pop_sorted)[,1:mu] - matrix(rep(m,mu), point_dim, mu)

-2.738318e-21,5.250223e-21,5.455235e-22
-2.047070e-20,2.803018e-20,3.110853e-20
-9.982478e-22,-4.684752e-21,1.609794e-20
